# Amazon Deep Insights: Initial Data Exploration

This notebook provides an initial exploration of LiDAR and ancillary datasets for the Amazon rainforest. We'll download sample data, visualize LiDAR point clouds, generate basic terrain models, and explore potential archaeological and ecological features.

## Contents
1. Setup Environment
2. Download Sample Data
3. LiDAR Data Visualization
4. Basic Terrain Analysis
5. Feature Detection Exploration

## 1. Setup Environment

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Standard libraries
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from dotenv import load_dotenv

# Geospatial libraries
import geopandas as gpd
import rasterio
from rasterio.plot import show as rshow
import folium
import pyproj
from shapely.geometry import Point, Polygon, box

# LiDAR specific
import laspy
import pdal
import json

# Visualization
from ipywidgets import interact, fixed
import ipywidgets as widgets
from IPython.display import display, HTML

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Load environment variables
load_dotenv()

# Add project root to path to import project modules
project_root = Path(os.getcwd()).parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Configure paths
RAW_DATA_DIR = os.environ.get('RAW_DATA_DIR', os.path.join(project_root, 'data', 'raw'))
PROCESSED_DATA_DIR = os.environ.get('PROCESSED_DATA_DIR', os.path.join(project_root, 'data', 'processed'))
LIDAR_TILES_DIR = os.environ.get('LIDAR_TILES_DIR', os.path.join(RAW_DATA_DIR, 'lidar'))

# Create directories if they don't exist
os.makedirs(RAW_DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(LIDAR_TILES_DIR, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Raw data directory: {RAW_DATA_DIR}")
print(f"Processed data directory: {PROCESSED_DATA_DIR}")

## 2. Download Sample Data

Let's download a small sample of LiDAR data from the ORNL DAAC repository. We'll use a sample from the "LiDAR Forest Inventory in Brazil" dataset (ds_id=1644).

In [ ]:
import requests
import shutil
from tqdm.notebook import tqdm

def download_file(url, local_path):
    """Download a file with progress bar"""
    if os.path.exists(local_path):
        print(f"File already exists at {local_path}")
        return local_path
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    
    # Stream download with progress bar
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(local_path, 'wb') as file, tqdm(
        desc=os.path.basename(local_path),
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
    
    return local_path

# For this example, we'll use a small sample LiDAR file
# Note: In a real implementation, we would fetch from the actual ORNL DAAC API
# but for demo purposes, we'll use a publicly accessible LiDAR sample

# Sample LiDAR file URL (this is a placeholder - replace with actual Amazon data URL)
sample_lidar_url = "https://github.com/PDAL/data/raw/master/autzen/autzen-classified.las"
sample_lidar_path = os.path.join(LIDAR_TILES_DIR, "sample", "sample_classified.las")

# Download the sample file
try:
    downloaded_file = download_file(sample_lidar_url, sample_lidar_path)
    print(f"Downloaded sample LiDAR file to: {downloaded_file}")
except Exception as e:
    print(f"Error downloading file: {e}")
    # If download fails, check if we have any LAS/LAZ files in the directory
    existing_files = list(Path(LIDAR_TILES_DIR).glob("**/*.la[sz]"))
    if existing_files:
        sample_lidar_path = str(existing_files[0])
        print(f"Using existing LiDAR file: {sample_lidar_path}")
    else:
        print("No LiDAR files found. Please download manually.")

### Download Supplementary Data

Let's also download a sample of known archaeological sites in the Amazon region to provide context for our analysis.

In [ ]:
# Create a simple GeoJSON with known archaeological sites
# In a real implementation, we would fetch this from a proper source
# This is just placeholder data for demonstration

known_sites = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "name": "Geoglyph Site 1",
                "type": "Geometric earthwork",
                "discovered": 2009
            },
            "geometry": {
                "type": "Point",
                "coordinates": [-67.4, -9.8]
            }
        },
        {
            "type": "Feature",
            "properties": {
                "name": "Geoglyph Site 2",
                "type": "Circular earthwork",
                "discovered": 2016
            },
            "geometry": {
                "type": "Point",
                "coordinates": [-67.5, -9.9]
            }
        },
        {
            "type": "Feature",
            "properties": {
                "name": "Terra Preta Site",
                "type": "Dark earth settlement",
                "discovered": 2005
            },
            "geometry": {
                "type": "Point",
                "coordinates": [-62.2, -3.4]
            }
        }
    ]
}

# Save to GeoJSON file
sites_dir = os.path.join(RAW_DATA_DIR, "vectors", "known_sites")
os.makedirs(sites_dir, exist_ok=True)
sites_path = os.path.join(sites_dir, "sample_sites.geojson")

with open(sites_path, 'w') as f:
    json.dump(known_sites, f)

print(f"Created sample archaeological sites GeoJSON at: {sites_path}")

# Load as GeoDataFrame for later use
sites_gdf = gpd.read_file(sites_path)
sites_gdf.head()

## 3. LiDAR Data Visualization

Now that we have our sample data, let's explore the LiDAR point cloud.

In [ ]:
# Load the LiDAR file using laspy
try:
    las = laspy.read(sample_lidar_path)
    print(f"LiDAR file loaded: {sample_lidar_path}")
    print(f"Number of points: {len(las.points)}")
    print(f"Point format: {las.header.point_format}")
    print(f"CRS: {las.header.parse_crs()}")
    
    # Print available dimensions
    print("\nAvailable dimensions:")
    for dim in las.point_format.dimensions:
        print(f"  - {dim.name}")
        
    # Print classification counts if available
    if hasattr(las, 'classification'):
        class_counts = pd.Series(las.classification).value_counts()
        print("\nClassification counts:")
        for cls, count in class_counts.items():
            # Standard LAS classification codes
            class_names = {
                0: "Created, never classified",
                1: "Unclassified",
                2: "Ground",
                3: "Low Vegetation",
                4: "Medium Vegetation",
                5: "High Vegetation",
                6: "Building",
                7: "Low Point (noise)",
                8: "Model Key-point",
                9: "Water",
                10: "Rail",
                11: "Road Surface",
                12: "Reserved",
                13: "Wire - Guard (Shield)",
                14: "Wire - Conductor (Phase)",
                15: "Transmission Tower",
                16: "Wire-structure Connector",
                17: "Bridge Deck",
                18: "High Noise"
            }
            class_name = class_names.get(cls, "Unknown")
            print(f"  - Class {cls} ({class_name}): {count} points ({count/len(las.points)*100:.2f}%)")
except Exception as e:
    print(f"Error loading LiDAR file: {e}")

### 3D Visualization of the Point Cloud

Let's create a 3D visualization of the point cloud using a sample of points.

In [ ]:
# Sample points to make visualization manageable
try:
    # Take a random sample of points (adjust sample_size as needed)
    sample_size = min(1000000, len(las.points))
    indices = np.random.choice(len(las.points), sample_size, replace=False)
    
    # Extract coordinates
    x = las.x[indices]
    y = las.y[indices]
    z = las.z[indices]
    
    # Normalize coordinates for better visualization
    x_norm = x - np.min(x)
    y_norm = y - np.min(y)
    z_norm = z - np.min(z)
    
    # Create color array based on classification if available
    if hasattr(las, 'classification'):
        # Use classification for coloring
        classifications = las.classification[indices]
        
        # Create a colormap for different classes
        cmap = plt.cm.get_cmap('viridis', len(np.unique(classifications)))
        colors = cmap(classifications / np.max(classifications))
    else:
        # Use height (z) for coloring if classification not available
        colors = plt.cm.viridis(z_norm / np.max(z_norm))
    
    # Create 3D scatter plot
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot points
    scatter = ax.scatter(x_norm, y_norm, z_norm, c=colors, s=0.1, alpha=0.5)
    
    # Add colorbar if using classification
    if hasattr(las, 'classification'):
        cbar = plt.colorbar(scatter)
        cbar.set_label('Classification')
    
    # Set labels and title
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title('LiDAR Point Cloud Visualization (Sample)')
    
    # Improve 3D viewing angle
    ax.view_init(elev=20, azim=225)
    
    plt.tight_layout()
    plt.show()
    
    # Add a note about plas.io for better visualization
    print("Note: For better interactive visualization of large point clouds, consider using plas.io")
    print("      or other specialized LiDAR viewers mentioned in the project links.")
except Exception as e:
    print(f"Error creating 3D visualization: {e}")

### Point Cloud Profile View

Let's create a profile view (cross-section) of the point cloud to better visualize the terrain and vegetation structure.

In [ ]:
# Create a profile view (cross-section) of the point cloud
try:
    # Define a slice through the middle of the point cloud
    y_mid = (np.min(y) + np.max(y)) / 2
    slice_width = (np.max(y) - np.min(y)) * 0.01  # 1% of total width
    
    # Select points within the slice
    slice_mask = (las.y > y_mid - slice_width) & (las.y < y_mid + slice_width)
    slice_x = las.x[slice_mask]
    slice_z = las.z[slice_mask]
    
    # If available, get classification for coloring
    if hasattr(las, 'classification'):
        slice_class = las.classification[slice_mask]
        
        # Create color map
        class_colors = {
            0: 'gray',      # Created, never classified
            1: 'lightgray', # Unclassified
            2: 'brown',     # Ground
            3: 'yellowgreen', # Low Vegetation
            4: 'green',     # Medium Vegetation
            5: 'darkgreen', # High Vegetation
            6: 'red',       # Building
            7: 'black',     # Low Point (noise)
            8: 'orange',    # Model Key-point
            9: 'blue',      # Water
        }
        
        # Default color for classes not in the map
        colors = np.array(['gray'] * len(slice_class))
        
        # Assign colors based on classification
        for cls, color in class_colors.items():
            colors[slice_class == cls] = color
    else:
        # Use height for coloring if classification not available
        colors = plt.cm.viridis((slice_z - np.min(slice_z)) / (np.max(slice_z) - np.min(slice_z)))
    
    # Create the profile plot
    plt.figure(figsize=(15, 6))
    
    # Plot points
    if hasattr(las, 'classification'):
        # Plot by classification groups for better legend
        for cls, color in class_colors.items():
            mask = slice_class == cls
            if np.any(mask):
                plt.scatter(slice_x[mask], slice_z[mask], c=color, s=1, alpha=0.7, 
                           label=f"Class {cls}")
        plt.legend(markerscale=5)
    else:
        plt.scatter(slice_x, slice_z, c=colors, s=1, alpha=0.7)
    
    # Add labels and title
    plt.xlabel('X (m)')
    plt.ylabel('Z (m) - Elevation')
    plt.title(f'LiDAR Point Cloud Profile (Y = {y_mid:.2f} ± {slice_width:.2f} m)')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error creating profile view: {e}")

## 4. Basic Terrain Analysis

Now let's use PDAL to generate a Digital Elevation Model (DEM) and a Canopy Height Model (CHM) from our LiDAR data.

In [ ]:
# Create output directories
dem_dir = os.path.join(PROCESSED_DATA_DIR, "dem")
chm_dir = os.path.join(PROCESSED_DATA_DIR, "chm")
os.makedirs(dem_dir, exist_ok=True)
os.makedirs(chm_dir, exist_ok=True)

# Output file paths
dem_path = os.path.join(dem_dir, "sample_dem.tif")
chm_path = os.path.join(chm_dir, "sample_chm.tif")

# Define PDAL pipeline for DEM generation
dem_pipeline = {
    "pipeline": [
        sample_lidar_path,
        {
            "type": "filters.assign",
            "assignment": "Classification[:]=0"
        },
        {
            "type": "filters.elm"
        },
        {
            "type": "filters.outlier",
            "method": "statistical",
            "mean_k": 8,
            "multiplier": 3.0
        },
        {
            "type": "filters.pmf"
        },
        {
            "type": "filters.range",
            "limits": "Classification[2:2]"
        },
        {
            "type": "writers.gdal",
            "filename": dem_path,
            "output_type": "idw",
            "resolution": 1.0,
            "window_size": 10
        }
    ]
}

# Execute the DEM pipeline
try:
    pipeline = pdal.Pipeline(json.dumps(dem_pipeline))
    pipeline.execute()
    print(f"DEM created successfully: {dem_path}")
except Exception as e:
    print(f"Error creating DEM: {e}")

In [ ]:
# Define PDAL pipeline for DSM (Digital Surface Model) generation
dsm_path = os.path.join(PROCESSED_DATA_DIR, "dsm", "sample_dsm.tif")
os.makedirs(os.path.dirname(dsm_path), exist_ok=True)

dsm_pipeline = {
    "pipeline": [
        sample_lidar_path,
        {
            "type": "filters.outlier",
            "method": "statistical",
            "mean_k": 8,
            "multiplier": 3.0
        },
        {
            "type": "writers.gdal",
            "filename": dsm_path,
            "output_type": "max",  # Use max elevation for DSM
            "resolution": 1.0,
            "window_size": 10
        }
    ]
}

# Execute the DSM pipeline
try:
    pipeline = pdal.Pipeline(json.dumps(dsm_pipeline))
    pipeline.execute()
    print(f"DSM created successfully: {dsm_path}")
    
    # Calculate CHM by subtracting DEM from DSM
    with rasterio.open(dem_path) as dem_src, rasterio.open(dsm_path) as dsm_src:
        dem_data = dem_src.read(1)
        dsm_data = dsm_src.read(1)
        
        # Calculate CHM (DSM - DEM)
        chm_data = dsm_data - dem_data
        
        # Set negative values to zero (sometimes happens due to interpolation artifacts)
        chm_data[chm_data < 0] = 0
        
        # Write CHM to file
        with rasterio.open(
            chm_path,
            'w',
            driver='GTiff',
            height=dem_src.height,
            width=dem_src.width,
            count=1,
            dtype=chm_data.dtype,
            crs=dem_src.crs,
            transform=dem_src.transform,
        ) as chm_dst:
            chm_dst.write(chm_data, 1)
        
        print(f"CHM created successfully: {chm_path}")
except Exception as e:
    print(f"Error creating DSM or CHM: {e}")

### Visualize Terrain Models

Let's visualize the DEM and CHM we just created.

In [ ]:
# Visualize DEM
try:
    with rasterio.open(dem_path) as src:
        dem_data = src.read(1, masked=True)
        
        fig, ax = plt.subplots(figsize=(12, 10))
        im = ax.imshow(dem_data, cmap='terrain')
        ax.set_title('Digital Elevation Model (DEM)')
        plt.colorbar(im, ax=ax, label='Elevation (m)')
        plt.tight_layout()
        plt.show()
except Exception as e:
    print(f"Error visualizing DEM: {e}")

# Visualize CHM
try:
    with rasterio.open(chm_path) as src:
        chm_data = src.read(1, masked=True)
        
        fig, ax = plt.subplots(figsize=(12, 10))
        im = ax.imshow(chm_data, cmap='viridis')
        ax.set_title('Canopy Height Model (CHM)')
        plt.colorbar(im, ax=ax, label='Height (m)')
        plt.tight_layout()
        plt.show()
        
        # Print some statistics about the canopy height
        valid_data = chm_data[~chm_data.mask]
        print(f"Canopy Height Statistics:")
        print(f"  Min: {np.min(valid_data):.2f} m")
        print(f"  Max: {np.max(valid_data):.2f} m")
        print(f"  Mean: {np.mean(valid_data):.2f} m")
        print(f"  Median: {np.median(valid_data):.2f} m")
        print(f"  Standard Deviation: {np.std(valid_data):.2f} m")
except Exception as e:
    print(f"Error visualizing CHM: {e}")

## 5. Feature Detection Exploration

Now let's explore some basic feature detection techniques that could help identify archaeological features or significant ecological structures (like giant trees).

In [ ]:
# Slope Analysis - can help identify artificial structures
try:
    with rasterio.open(dem_path) as src:
        dem = src.read(1, masked=True)
        transform = src.transform
        
        # Calculate x and y gradients
        dx, dy = np.gradient(dem)
        
        # Calculate slope in degrees
        slope = np.degrees(np.arctan(np.sqrt(dx**2 + dy**2)))
        
        # Calculate aspect (direction of slope)
        aspect = np.degrees(np.arctan2(-dx, dy))
        
        # Visualize slope
        fig, ax = plt.subplots(figsize=(12, 10))
        im = ax.imshow(slope, cmap='YlOrRd')
        ax.set_title('Slope (degrees)')
        plt.colorbar(im, ax=ax, label='Slope (degrees)')
        plt.tight_layout()
        plt.show()
        
        # Visualize aspect
        fig, ax = plt.subplots(figsize=(12, 10))
        im = ax.imshow(aspect, cmap='hsv')
        ax.set_title('Aspect (degrees)')
        plt.colorbar(im, ax=ax, label='Aspect (degrees)')
        plt.tight_layout()
        plt.show()
except Exception as e:
    print(f"Error calculating slope and aspect: {e}")

In [ ]:
# Basic tree detection using local maxima in the CHM
try:
    from skimage.feature import peak_local_max
    from scipy import ndimage
    
    with rasterio.open(chm_path) as src:
        chm = src.read(1, masked=True)
        transform = src.transform
        
        # Fill masked values with 0
        chm_filled = chm.filled(0)
        
        # Apply Gaussian filter to smooth the CHM
        chm_smooth = ndimage.gaussian_filter(chm_filled, sigma=1)
        
        # Find local maxima (potential tree tops)
        # Adjust min_distance and threshold based on expected tree spacing and minimum height
        min_height = 5  # Minimum tree height in meters
        min_distance = 5  # Minimum distance between trees in pixels
        
        # Find peaks
        tree_tops = peak_local_max(
            chm_smooth, 
            min_distance=min_distance,
            threshold_abs=min_height,
            indices=True
        )
        
        # Get tree heights at peak locations
        tree_heights = chm_smooth[tree_tops[:, 0], tree_tops[:, 1]]
        
        # Create a visualization
        fig, ax = plt.subplots(figsize=(12, 10))
        im = ax.imshow(chm_smooth, cmap='viridis')
        ax.plot(tree_tops[:, 1], tree_tops[:, 0], 'r.', markersize=3)
        ax.set_title(f'Tree Top Detection (n={len(tree_tops)})')
        plt.colorbar(im, ax=ax, label='Height (m)')
        plt.tight_layout()
        plt.show()
        
        # Print statistics about detected trees
        print(f"Tree Detection Results:")
        print(f"  Number of trees detected: {len(tree_heights)}")
        print(f"  Mean tree height: {np.mean(tree_heights):.2f} m")
        print(f"  Max tree height: {np.max(tree_heights):.2f} m")
        print(f"  Min tree height: {np.min(tree_heights):.2f} m")
        
        # Find potential giant trees (e.g., top 5% by height)
        height_threshold = np.percentile(tree_heights, 95)
        giant_trees = tree_tops[tree_heights > height_threshold]
        giant_tree_heights = tree_heights[tree_heights > height_threshold]
        
        print(f"\nPotential Giant Trees (height > {height_threshold:.2f} m):")
        print(f"  Number detected: {len(giant_trees)}")
        if len(giant_trees) > 0:
            print(f"  Mean height: {np.mean(giant_tree_heights):.2f} m")
            print(f"  Max height: {np.max(giant_tree_heights):.2f} m")
            
            # Highlight giant trees in a new visualization
            fig, ax = plt.subplots(figsize=(12, 10))
            im = ax.imshow(chm_smooth, cmap='viridis')
            ax.plot(tree_tops[:, 1], tree_tops[:, 0], 'r.', markersize=2, alpha=0.5)
            ax.plot(giant_trees[:, 1], giant_trees[:, 0], 'yo', markersize=8, mfc='none')
            ax.set_title(f'Potential Giant Trees (n={len(giant_trees)})')
            plt.colorbar(im, ax=ax, label='Height (m)')
            plt.tight_layout()
            plt.show()
except Exception as e:
    print(f"Error in tree detection: {e}")

## Conclusion and Next Steps

In this notebook, we've explored the basics of working with LiDAR data for the Amazon Deep Insights project. We've:

1. Set up our environment and downloaded sample data
2. Visualized LiDAR point clouds in 3D and profile views
3. Generated Digital Elevation Models (DEM) and Canopy Height Models (CHM)
4. Performed basic terrain analysis (slope, aspect)
5. Explored simple tree detection techniques

### Next Steps

1. **Data Acquisition**: Download actual Amazon rainforest LiDAR data from the sources listed in Links.md
2. **Advanced Processing**: Implement more sophisticated filtering and classification techniques
3. **Feature Detection**: Develop algorithms to detect potential archaeological features (geometric patterns, mounds)
4. **RAG Integration**: Connect the processed data to our RAG system for natural language querying
5. **Visualization**: Create interactive maps and dashboards for exploration

The sample data used in this notebook is for demonstration purposes only. For the actual project, we'll need to work with the Amazon-specific datasets listed in the project resources.